In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\nickv\anaconda3\envs\pythonadvanced2\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.

# 'koi_period','koi_time0bk','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr',
# 'koi_tce_plnt_num','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag'

selected_features = df[['koi_period','koi_time0bk','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr','koi_tce_plnt_num','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split

X = selected_features
y = df["koi_disposition"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print("Shape: ", X.shape, y.shape)


Shape:  (6991, 16) (6991,)


In [7]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,1,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,1,5855,4.578,0.797,284.50391,42.463860,15.770
2879,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,1,6328,4.481,0.963,295.50211,38.983540,13.099
107,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,1,4768,4.536,0.779,291.15878,40.750271,15.660
29,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,1,5712,4.359,1.082,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)



In [9]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=8, activation='relu', input_dim=16))
deep_model.add(Dense(units=8, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))

deep_model.summary()

deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)


model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 136       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
Total params: 235
Trainable params: 235
Non-trainable params: 0
_________________________________________________________________
Train on 5243 samples
Epoch 1/1000
5243/5243 - 1s - loss: 1.0316 - accuracy: 0.5056
Epoch 2/1000
5243/5243 - 0s - loss: 1.0129 - accuracy: 0.5056
Epoch 3/1000
5243/5243 - 0s - loss: 0.9723 - accuracy: 0.5056
Epoch 4/1000
5243/5243 - 0s - loss: 0.9223 - accuracy: 0.5360
Epoch 5/1000
5243/5243 - 0s - loss: 0.8887 - accuracy: 0.5789
Epoch 6/1000
5243/5243 - 0s - loss: 0.8660 - accuracy: 0

Epoch 119/1000
5243/5243 - 0s - loss: 0.6781 - accuracy: 0.7061
Epoch 120/1000
5243/5243 - 0s - loss: 0.6801 - accuracy: 0.7027
Epoch 121/1000
5243/5243 - 0s - loss: 0.6760 - accuracy: 0.7067
Epoch 122/1000
5243/5243 - 0s - loss: 0.6753 - accuracy: 0.7067
Epoch 123/1000
5243/5243 - 0s - loss: 0.6752 - accuracy: 0.7082
Epoch 124/1000
5243/5243 - 0s - loss: 0.6762 - accuracy: 0.7059
Epoch 125/1000
5243/5243 - 0s - loss: 0.6749 - accuracy: 0.7047
Epoch 126/1000
5243/5243 - 0s - loss: 0.6744 - accuracy: 0.7080
Epoch 127/1000
5243/5243 - 0s - loss: 0.6725 - accuracy: 0.7063
Epoch 128/1000
5243/5243 - 0s - loss: 0.6728 - accuracy: 0.7044
Epoch 129/1000
5243/5243 - 0s - loss: 0.6759 - accuracy: 0.7047
Epoch 130/1000
5243/5243 - 0s - loss: 0.6734 - accuracy: 0.7084
Epoch 131/1000
5243/5243 - 0s - loss: 0.6743 - accuracy: 0.7023
Epoch 132/1000
5243/5243 - 0s - loss: 0.6708 - accuracy: 0.7067
Epoch 133/1000
5243/5243 - 0s - loss: 0.6703 - accuracy: 0.7082
Epoch 134/1000
5243/5243 - 0s - loss: 0.

Epoch 248/1000
5243/5243 - 0s - loss: 0.6224 - accuracy: 0.7379
Epoch 249/1000
5243/5243 - 0s - loss: 0.6216 - accuracy: 0.7368
Epoch 250/1000
5243/5243 - 0s - loss: 0.6216 - accuracy: 0.7341
Epoch 251/1000
5243/5243 - 0s - loss: 0.6227 - accuracy: 0.7330
Epoch 252/1000
5243/5243 - 0s - loss: 0.6208 - accuracy: 0.7358
Epoch 253/1000
5243/5243 - 0s - loss: 0.6191 - accuracy: 0.7391
Epoch 254/1000
5243/5243 - 0s - loss: 0.6192 - accuracy: 0.7385
Epoch 255/1000
5243/5243 - 0s - loss: 0.6208 - accuracy: 0.7385
Epoch 256/1000
5243/5243 - 0s - loss: 0.6185 - accuracy: 0.7376
Epoch 257/1000
5243/5243 - 0s - loss: 0.6193 - accuracy: 0.7332
Epoch 258/1000
5243/5243 - 0s - loss: 0.6192 - accuracy: 0.7374
Epoch 259/1000
5243/5243 - 0s - loss: 0.6169 - accuracy: 0.7381
Epoch 260/1000
5243/5243 - 0s - loss: 0.6206 - accuracy: 0.7383
Epoch 261/1000
5243/5243 - 0s - loss: 0.6188 - accuracy: 0.7345
Epoch 262/1000
5243/5243 - 0s - loss: 0.6168 - accuracy: 0.7421
Epoch 263/1000
5243/5243 - 0s - loss: 0.

5243/5243 - 0s - loss: 0.5910 - accuracy: 0.7549
Epoch 377/1000
5243/5243 - 0s - loss: 0.5921 - accuracy: 0.7540
Epoch 378/1000
5243/5243 - 0s - loss: 0.5922 - accuracy: 0.7540
Epoch 379/1000
5243/5243 - 0s - loss: 0.5893 - accuracy: 0.7572
Epoch 380/1000
5243/5243 - 0s - loss: 0.5929 - accuracy: 0.7545
Epoch 381/1000
5243/5243 - 0s - loss: 0.5892 - accuracy: 0.7599
Epoch 382/1000
5243/5243 - 0s - loss: 0.5867 - accuracy: 0.7566
Epoch 383/1000
5243/5243 - 0s - loss: 0.5877 - accuracy: 0.7608
Epoch 384/1000
5243/5243 - 0s - loss: 0.5908 - accuracy: 0.7509
Epoch 385/1000
5243/5243 - 0s - loss: 0.5893 - accuracy: 0.7541
Epoch 386/1000
5243/5243 - 0s - loss: 0.5860 - accuracy: 0.7578
Epoch 387/1000
5243/5243 - 0s - loss: 0.5890 - accuracy: 0.7572
Epoch 388/1000
5243/5243 - 0s - loss: 0.5877 - accuracy: 0.7551
Epoch 389/1000
5243/5243 - 0s - loss: 0.5867 - accuracy: 0.7618
Epoch 390/1000
5243/5243 - 0s - loss: 0.5884 - accuracy: 0.7543
Epoch 391/1000
5243/5243 - 0s - loss: 0.5894 - accuracy

Epoch 505/1000
5243/5243 - 0s - loss: 0.5745 - accuracy: 0.7603
Epoch 506/1000
5243/5243 - 0s - loss: 0.5774 - accuracy: 0.7601
Epoch 507/1000
5243/5243 - 0s - loss: 0.5837 - accuracy: 0.7547
Epoch 508/1000
5243/5243 - 0s - loss: 0.5763 - accuracy: 0.7618
Epoch 509/1000
5243/5243 - 0s - loss: 0.5768 - accuracy: 0.7618
Epoch 510/1000
5243/5243 - 0s - loss: 0.5758 - accuracy: 0.7627
Epoch 511/1000
5243/5243 - 0s - loss: 0.5756 - accuracy: 0.7593
Epoch 512/1000
5243/5243 - 0s - loss: 0.5762 - accuracy: 0.7606
Epoch 513/1000
5243/5243 - 0s - loss: 0.5760 - accuracy: 0.7629
Epoch 514/1000
5243/5243 - 0s - loss: 0.5769 - accuracy: 0.7593
Epoch 515/1000
5243/5243 - 0s - loss: 0.5760 - accuracy: 0.7601
Epoch 516/1000
5243/5243 - 0s - loss: 0.5777 - accuracy: 0.7606
Epoch 517/1000
5243/5243 - 0s - loss: 0.5749 - accuracy: 0.7595
Epoch 518/1000
5243/5243 - 0s - loss: 0.5720 - accuracy: 0.7620
Epoch 519/1000
5243/5243 - 0s - loss: 0.5753 - accuracy: 0.7623
Epoch 520/1000
5243/5243 - 0s - loss: 0.

Epoch 634/1000
5243/5243 - 0s - loss: 0.5650 - accuracy: 0.7601
Epoch 635/1000
5243/5243 - 0s - loss: 0.5711 - accuracy: 0.7608
Epoch 636/1000
5243/5243 - 0s - loss: 0.5682 - accuracy: 0.7650
Epoch 637/1000
5243/5243 - 0s - loss: 0.5656 - accuracy: 0.7665
Epoch 638/1000
5243/5243 - 0s - loss: 0.5690 - accuracy: 0.7616
Epoch 639/1000
5243/5243 - 0s - loss: 0.5677 - accuracy: 0.7662
Epoch 640/1000
5243/5243 - 0s - loss: 0.5685 - accuracy: 0.7625
Epoch 641/1000
5243/5243 - 0s - loss: 0.5687 - accuracy: 0.7604
Epoch 642/1000
5243/5243 - 0s - loss: 0.5684 - accuracy: 0.7620
Epoch 643/1000
5243/5243 - 0s - loss: 0.5667 - accuracy: 0.7637
Epoch 644/1000
5243/5243 - 0s - loss: 0.5684 - accuracy: 0.7637
Epoch 645/1000
5243/5243 - 0s - loss: 0.5659 - accuracy: 0.7631
Epoch 646/1000
5243/5243 - 0s - loss: 0.5669 - accuracy: 0.7646
Epoch 647/1000
5243/5243 - 0s - loss: 0.5675 - accuracy: 0.7646
Epoch 648/1000
5243/5243 - 0s - loss: 0.5670 - accuracy: 0.7631
Epoch 649/1000
5243/5243 - 0s - loss: 0.

5243/5243 - 0s - loss: 0.5599 - accuracy: 0.7671
Epoch 763/1000
5243/5243 - 0s - loss: 0.5606 - accuracy: 0.7667
Epoch 764/1000
5243/5243 - 0s - loss: 0.5632 - accuracy: 0.7664
Epoch 765/1000
5243/5243 - 0s - loss: 0.5639 - accuracy: 0.7662
Epoch 766/1000
5243/5243 - 0s - loss: 0.5632 - accuracy: 0.7664
Epoch 767/1000
5243/5243 - 0s - loss: 0.5601 - accuracy: 0.7667
Epoch 768/1000
5243/5243 - 0s - loss: 0.5584 - accuracy: 0.7690
Epoch 769/1000
5243/5243 - 0s - loss: 0.5615 - accuracy: 0.7644
Epoch 770/1000
5243/5243 - 0s - loss: 0.5632 - accuracy: 0.7646
Epoch 771/1000
5243/5243 - 0s - loss: 0.5630 - accuracy: 0.7652
Epoch 772/1000
5243/5243 - 0s - loss: 0.5630 - accuracy: 0.7631
Epoch 773/1000
5243/5243 - 0s - loss: 0.5641 - accuracy: 0.7641
Epoch 774/1000
5243/5243 - 0s - loss: 0.5620 - accuracy: 0.7627
Epoch 775/1000
5243/5243 - 0s - loss: 0.5636 - accuracy: 0.7644
Epoch 776/1000
5243/5243 - 0s - loss: 0.5582 - accuracy: 0.7667
Epoch 777/1000
5243/5243 - 0s - loss: 0.5619 - accuracy

Epoch 891/1000
5243/5243 - 0s - loss: 0.5576 - accuracy: 0.7664
Epoch 892/1000
5243/5243 - 0s - loss: 0.5557 - accuracy: 0.7685
Epoch 893/1000
5243/5243 - 0s - loss: 0.5575 - accuracy: 0.7641
Epoch 894/1000
5243/5243 - 0s - loss: 0.5571 - accuracy: 0.7660
Epoch 895/1000
5243/5243 - 0s - loss: 0.5578 - accuracy: 0.7648
Epoch 896/1000
5243/5243 - 0s - loss: 0.5601 - accuracy: 0.7604
Epoch 897/1000
5243/5243 - 0s - loss: 0.5570 - accuracy: 0.7633
Epoch 898/1000
5243/5243 - 0s - loss: 0.5575 - accuracy: 0.7623
Epoch 899/1000
5243/5243 - 0s - loss: 0.5561 - accuracy: 0.7686
Epoch 900/1000
5243/5243 - 0s - loss: 0.5564 - accuracy: 0.7652
Epoch 901/1000
5243/5243 - 0s - loss: 0.5633 - accuracy: 0.7667
Epoch 902/1000
5243/5243 - 0s - loss: 0.5603 - accuracy: 0.7637
Epoch 903/1000
5243/5243 - 0s - loss: 0.5548 - accuracy: 0.7650
Epoch 904/1000
5243/5243 - 0s - loss: 0.5559 - accuracy: 0.7667
Epoch 905/1000
5243/5243 - 0s - loss: 0.5570 - accuracy: 0.7654
Epoch 906/1000
5243/5243 - 0s - loss: 0.

# Save the Model

In [10]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

#import joblib
#filename = 'nv_deep.sav'
#joblib.dump(deep_model, filename)

deep_model.save("nv_deep.h5")